In [4]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
sc=SparkContext()

In [6]:
#import SQLContext and crete a sqlcontext from SparkContext
from  pyspark import SQLContext
sqlContext=SQLContext(sc)

In [7]:

#............................................................................
##   Working with Data Frames
#............................................................................


In [8]:
#Create a DataFrame with empautoData=sc.LoadFile()

empDF=sqlContext.read.json("customerData.json")
empDF

DataFrame[age: string, deptid: string, gender: string, name: string, salary: string]

In [9]:
empDF.show()# show content of Dataframe
empDF.printSchema()# print schema of Dataframe

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



In [10]:
#do SQL queries as Data Frame
empDF.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



In [11]:
empDF.select(["name","salary"]).show()# more than one columns

+-----------------+------+
|             name|salary|
+-----------------+------+
|Benjamin Garrison|  3000|
|    Holland Drake|  4500|
|  Burks Velasquez|  2700|
|    June Rutledge|  4300|
|    Nielsen Knapp|  6500|
+-----------------+------+



In [12]:
empDF.select(["name","salary"]).show()

+-----------------+------+
|             name|salary|
+-----------------+------+
|Benjamin Garrison|  3000|
|    Holland Drake|  4500|
|  Burks Velasquez|  2700|
|    June Rutledge|  4300|
|    Nielsen Knapp|  6500|
+-----------------+------+



In [13]:
#to evaluate a condion
empDF.select(empDF["age"]==40).show()

+----------+
|(age = 40)|
+----------+
|     false|
|      true|
|     false|
|     false|
|     false|
+----------+



In [14]:
#filter the resultset
empDF.filter(empDF["age"]==40).show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



In [16]:
empDF.show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



In [18]:
empDF.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    4|
+------+-----+



In [23]:
genderGrp=(empDF.groupBy("gender").count())
genderGrp.filter(genderGrp["gender"]=="male").show()

+------+-----+
|gender|count|
+------+-----+
|  male|    4|
+------+-----+



In [25]:
empDF.groupBy("deptid").agg({"salary":"avg","age":"max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



In [33]:
#create a data frame from a list
deptList = [{'name': 'Sales', 'id': "100"},{ 'name':'Engineering','id':"200" }]

In [36]:
deptDf=sqlContext.createDataFrame(deptList)

C:\Users\Kishan\spark\spark-3.0.0-bin-hadoop2.7\python\pyspark\sql\session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [37]:
deptDf.show()

+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



In [40]:
#join the data frames
empDF.join(deptDf,empDF.deptid==deptDf.id).show()

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 51|   100|female|    June Rutledge|  4300|100|      Sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      Sales|
| 32|   100|  male|Benjamin Garrison|  3000|100|      Sales|
| 44|   200|  male|    Nielsen Knapp|  6500|200|Engineering|
| 40|   200|  male|    Holland Drake|  4500|200|Engineering|
+---+------+------+-----------------+------+---+-----------+



In [50]:
#cascading operations

empDF.filter(empDF["age"]>30).join(deptDf,empDF.deptid==deptDf.id).groupBy("deptid").agg({"salary": "avg", "age": "max"}).show()

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



In [51]:
#............................................................................
##   Creating and working with Temp Tables
#.

In [52]:
empDF.registerTempTable("emp")

In [55]:
sqlContext.sql("select * from emp").show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



In [63]:
sqlContext.sql("select * from emp where name='Benjamin Garrison' ").show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
+---+------+------+-----------------+------+



In [64]:
sqlContext.sql("select * from emp where salary>300 ").show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



In [65]:
#to Pandas DataFrame
empPandas=empDF.toPandas()

In [66]:
empPandas.head()

,age,deptid,gender,name,salary
0,32,100,male,Benjamin Garrison,3000
1,40,200,male,Holland Drake,4500
2,26,100,male,Burks Velasquez,2700
3,51,100,female,June Rutledge,4300
4,44,200,male,Nielsen Knapp,6500


In [67]:
#............................................................................
##   Working with Databases
#............................................................................
#Make sure that the spark classpaths are set appropriately in the 
#spark-defaults.conf file to include the driver files
    

In [ ]:
demoDF=sqlContext.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/demo",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "demotable",
    user="root",
    password="").load()

In [69]:
#............................................................................
##   Creating data frames from RDD
#............................................................................


In [70]:
from pyspark.sql import Row

In [71]:
lines=sc.textFile("auto-data.csv")
#remove the first line
datalines=lines.filter(lambda x : "FUELTYPE" not in x)
datalines.count()

197

In [72]:
data=datalines.map(lambda l:l.split(","))
autoMap=data.map(lambda l: Row(make=l[0],body=l[4],hp=int(l[7])))
autoMap.collect()

[Row(make='subaru', body='hatchback', hp=69),
 Row(make='chevrolet', body='hatchback', hp=48),
 Row(make='mazda', body='hatchback', hp=68),
 Row(make='toyota', body='hatchback', hp=62),
 Row(make='mitsubishi', body='hatchback', hp=68),
 Row(make='honda', body='hatchback', hp=60),
 Row(make='nissan', body='sedan', hp=69),
 Row(make='dodge', body='hatchback', hp=68),
 Row(make='plymouth', body='hatchback', hp=68),
 Row(make='mazda', body='hatchback', hp=68),
 Row(make='mitsubishi', body='hatchback', hp=68),
 Row(make='dodge', body='hatchback', hp=68),
 Row(make='plymouth', body='hatchback', hp=68),
 Row(make='chevrolet', body='hatchback', hp=70),
 Row(make='toyota', body='hatchback', hp=62),
 Row(make='dodge', body='hatchback', hp=68),
 Row(make='honda', body='hatchback', hp=58),
 Row(make='toyota', body='hatchback', hp=62),
 Row(make='honda', body='hatchback', hp=76),
 Row(make='chevrolet', body='sedan', hp=70),
 Row(make='nissan', body='sedan', hp=69),
 Row(make='mitsubishi', body='hat

In [75]:
#infer the schema and register DataFranme as table
autoDF=sqlContext.createDataFrame(autoMap)
autoDF.createOrReplaceTempView("autos")
sqlContext.sql("select * from autos where make='dodge' ").show()

+-----+---------+---+
| make|     body| hp|
+-----+---------+---+
|dodge|hatchback| 68|
|dodge|hatchback| 68|
|dodge|hatchback| 68|
|dodge|    sedan| 68|
|dodge|    sedan| 68|
|dodge|hatchback|102|
|dodge|    wagon| 88|
|dodge|hatchback|145|
+-----+---------+---+



In [77]:
sqlContext.sql("select * from autos where make='dodge' ").groupBy("body").count().show()

+---------+-----+
|     body|count|
+---------+-----+
|    wagon|    1|
|    sedan|    2|
|hatchback|    5|
+---------+-----+

